In [1]:
import requests
import warnings

from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM
warnings.simplefilter("ignore", FutureWarning)

model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large-ft", trust_remote_code=True, device_map="cuda").eval()
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large-ft", trust_remote_code=True)

/playpen-storage/levlevi/anaconda3/envs/vish/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/playpen-storage/levlevi/anaconda3/envs/vish/lib/python3.8/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


# **OCR Benchmark**
***

In [2]:
from ultralytics import YOLO
from PIL import Image
from glob import glob
import ast
import sys

THRESH = 0.00
yolo_model = YOLO("/playpen-storage/levlevi/player-re-id/src/testing/ocr_model_comparisons/text_detection/runs/detect/train4/weights/best.pt")

def det_bbox(image: Image): 
    # idk if this will work
    results = yolo_model([image], verbose=False)
    for _, result in enumerate(results):
        x1_min = sys.maxsize
        x2_max = 0
        y1_min = sys.maxsize
        y2_max = 0
        json_obj = ast.literal_eval(result.tojson())
        if len(json_obj) == 0:
            return None, None, None, None
        for pred in json_obj:
            if pred.get('confidence') < THRESH:
                continue
            x1, y1, x2, y2 = [pred.get('box').get(k) for k in pred.get('box')]
            if x1 < x1_min:
                x1_min = x1
            if x2 > x2_max:
                x2_max = x2
            if y1 < y1_min:
                y1_min = y1
            if y2 > y2_max:
                y2_max = y2
        if x2_max == 0:
            return None, None, None, None
        else:
            return int(x1_min), int(y1_min), int(x2_max), int(y2_max)
    return None, None, None, None

In [3]:
import os
from tqdm import tqdm
from glob import glob

test_set_dir = "/playpen-storage/levlevi/player-re-id/src/testing/ocr_model_comparisons/text_recognition/nba_100_test_set"
image_file_paths = glob(os.path.join(test_set_dir, '*.jpg'))
PROMPT = """Analyze the basketball player shown in the provided still tracklet frame and describe the following details:
1. Jersey Number: Identify the number on the player's jersey. If the player has no jersey, provide None.
Based on the frame description, produce an output prediction in the following JSON format:
{
  "jersey_number": "<predicted_jersey_number>",
}
[EOS]"""
# PROMPT="<OCR>"

def crop_image(image: Image):
    x1, y1, x2, y2 = det_bbox(Image.open(image_file_path))
    if x1 is None:
        return None
    cropped_img = image.crop((x1, y1, x2, y2))
    cropped_img.save('cropped_img.png')
    return cropped_img

def perform_ocr(image_file_path: str) -> str:
    image = Image.open(image_file_path)
    # image = crop_image(image)
    if not image:
        return None
    inputs = processor(text=PROMPT, images=image, return_tensors="pt").to('cuda')
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        do_sample=True,
        num_beams=10
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(generated_text, task="<OCR>", image_size=(image.width, image.height))
    return parsed_answer

ground_truth_labels = []
results = []
for image_file_path in tqdm(image_file_paths):
    # get human annotation
    ground_truth_label = image_file_path.split('/')[-1].split('_')[1].split('.')[0]
    ground_truth_labels.append(ground_truth_label)
    # perform ocr
    result = perform_ocr(image_file_path)
    results.append(result)

100%|██████████| 100/100 [00:53<00:00,  1.86it/s]


In [4]:
import pandas as pd
import re

ground_truth_labels = [f.split('/')[-1].split('_')[1].split('.')[0] for f in image_file_paths]
predictions = []
for r in results:
    if not r:
        pred = 'nan'
    else:
        pred = r['<OCR>']
    # match only numerical strings
    pred = re.sub('[^0-9]', '', pred)
    if pred == 'unanswerable' or pred == '':
        pred = 'nan'
    predictions.append(pred)
    
df = pd.DataFrame({'ground_truth_label': ground_truth_labels, 'prediction': predictions})
correct = df['ground_truth_label'] == df['prediction']
df['correct'] = correct
df.head()

,ground_truth_label,prediction,correct
0,na,nan,False
1,na,1,False
2,na,nan,False
3,2,2,True
4,36,nan,False


In [5]:
tp, fp = [], []
tn, fn = [], []
for row_dict in df.itertuples():
    row = list(row_dict)
    if row[-1] == True:
        tp.append(1)
    elif row[1] == 'na' and row[2] == 'nan':
        tn.append(1)
    elif row[1] == 'na' and row[2] != 'nan':
        fp.append(1)
    else:
        fn.append(1)
        
precision = sum(tp) / (sum(tp) + sum(fp))
recall = sum(tp) / (sum(tp) + sum(fn))

print(sum(tp), '/', sum(tp) + sum(fp))
print(sum(tp), '/', sum(tp) + sum(fn))

26 / 36
26 / 51


In [8]:
26/36

0.7222222222222222

In [6]:
# precision = tp / (tp + fp)

# **Race Identification Benchmark**
***

In [7]:
import os
from tqdm import tqdm
from glob import glob

test_set_dir = '/mnt/opr/levlevi/player-re-id/src/testing/ocr_model_comparions/nba_100_test_set'
image_file_paths = glob(os.path.join(test_set_dir, '*.jpg'))
PROMPT = """Analyze the basketball player shown in the provided still tracklet frame and describe the following details:
1. Player Race: Identify the players skin color as either 'black', 'white', 'mixed', or 'asian'.
Based on the frame description, produce an output prediction in the following JSON format:
{
  "race": "<predicted_race>",
}
[EOS]"""

def perform_ocr(image_file_path: str) -> str:
    image = Image.open(image_file_path)
    inputs = processor(text=PROMPT, images=image, return_tensors="pt").to('cuda')
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        do_sample=False,
        num_beams=3
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(generated_text, task="<CAPTION_TO_PHRASE_GROUNDING>", image_size=(image.width, image.height))
    return parsed_answer

ground_truth_labels = []
results = []
for image_file_path in tqdm(image_file_paths):
    # get human annotation
    ground_truth_label = image_file_path.split('/')[-1].split('_')[1].split('.')[0]
    ground_truth_labels.append(ground_truth_label)
    # perform ocr
    result = perform_ocr(image_file_path)
    results.append(result)

0it [00:00, ?it/s]
